# Практические работы
## 3. семантический поиск по документам из Object Storage ##
### Задача: "Разработать семантический поиск с механизмом контроля возвращаемых результатов" ###
#### Этап 1: Подготовка векторной базы данных ####
Цель: Преобразовать доверенные документы из Object Storage в структуру, пригодную для семантического поиска — векторное хранилище.
Что происходит в проекте:
1. ##### Загрузка документов из Yandex Object Storage: #####

In [ ]:
s3 = boto3.client(
    's3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id=os.getenv('S3_ACCESS_KEY'),
    aws_secret_access_key=os.getenv('S3_SECRET_KEY')
)

    Скачиваются все файлы из бакета rag-docs-trusted и папки docs/.
2. ##### Загрузка и парсинг документов #####
    Для каждого файла применяется соответствующий загрузчик:

In [ ]:
if path.endswith(".pdf"):
    loader = PyPDFLoader(path)
elif path.endswith(".txt"):
    loader = TextLoader(path, encoding="utf-8")
loaded = loader.load()

    Например, инструкция.pdf → извлекается текст → создаётся список объектов Document.
3. #### Фильтрация и валидация ####
    Удаляются документы с None или пустым контентом:

In [ ]:
valid_docs = [
    doc for doc in loaded
    if hasattr(doc, 'page_content') and
       isinstance(doc.page_content, str) and
       doc.page_content.strip()
]

    Защита от битых или пустых файлов.
4. ##### Разбиение на чанки #####
    Используется RecursiveCharacterTextSplitter:

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)
chunks = text_splitter.split_documents(docs)

4. Создание FAISS-индекса

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("./vectorstore_faiss")

    Каждый чанк преобразуется в вектор → сохраняется локально для быстрого поиска.
#### Этап 2: Реализация и настройка поискового движка ####
Цель: На основе пользовательского запроса найти наиболее релевантные фрагменты из векторной базы.
1. Загрузка индекса при запросе

In [ ]:
vectorstore = FAISS.load_local("./vectorstore_faiss", embeddings, allow_dangerous_deserialization=True)

    При первом запросе — если индекса нет, он создаётся из Object Storage.
2. Настройка ретривера

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    Возвращает топ-3 самых релевантных чанка по косинусному сходству.
3. Выполнение поиска

In [ ]:
retrieved_docs = retriever.invoke(current_user_input)

    Например, запрос: “Кто может получить доступ к системе?”
    Система находит чанк: “Доступ к системе разрешён только авторизованным пользователям.”
4. Формирование контекста

In [ ]:
context_chunks = "\n\n".join([doc.page_content for doc in retrieved_docs])

    Объединяет найденные фрагменты в одну строку для подстановки в промпт.
5. Логирование и защита

In [ ]:
if valid_contents:
    context_chunks = "\n\n".join(valid_contents)
    print(f"RAG: найдено {len(valid_contents)} релевантных фрагментов.")

    В терминале видно: сколько фрагментов найдено, ошибки не приводят к падению.

##### Пример поиска: #####
    Запрос пользователя: “Какие требования к доступу?” <br>
    Система находит: “Доступ к системе разрешён только авторизованным пользователям.” 
##### Лог в терминале: #####
    RAG: найдено 1 релевантных фрагментов.
Поиск семантический — не требуется точное совпадение слов.
#### Этап 3: Интеграция с LLM и валидация ответов ####
Цель: Отправить запрос + контекст в YandexGPT и получить точный, основанный на документах ответ.
1. ##### Формирование промпта в формате YandexGPT #####
    YandexGPT использует формат инструкций и альтернативных ролей — role: system, role: user.

In [ ]:
messages = [
    {
        "role": "system",
        "text": (
            "Ты — корпоративный ассистент. Отвечай строго по документам. "
            "Если информации нет — скажи 'В документах не указано'.\n\n"
            f"Контекст из документов:\n{context_text}"
        )
    },
    {
        "role": "user",
        "text": user_query
    }
]

Пример промпта:

In [ ]:
[
  {
    "role": "system",
    "text": "Ты — корпоративный ассистент... Контекст: Доступ к системе предоставляется только сотрудникам с действующей электронной подписью."
  },
  {
    "role": "user",
    "text": "Как получить доступ к системе?"
  }
]

2. Отправка в YandexGPT<br>
    Код вызова API:

In [ ]:
def ask_yandexgpt(messages: list, api_key: str, folder_id: str) -> str:
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Authorization": f"Api-Key {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "modelUri": f"gpt://{folder_id}/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.3,
            "maxTokens": 1000
        },
        "messages": messages
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result["result"]["alternatives"][0]["message"]["text"]
    else:
        return f"Ошибка YandexGPT: {response.text}"

3. ##### Пример ответа от YandexGPT #####
Запрос:
“Как получить доступ к системе?”<br>
Контекст:
“Доступ к системе предоставляется только сотрудникам с действующей электронной подписью.” <br>
Ответ модели: “Для получения доступа к системе вам необходимо быть сотрудником компании и иметь действующую электронную подпись.” <br>

Ответ основан на документе, без галлюцинаций.

4. ##### Валидация и постобработка #####

In [ ]:
if not answer.strip():
    answer = "Не удалось сгенерировать ответ."
# Можно добавить проверку на наличие ключевых слов из контекста

5. ##### Безопасность #####
    1. Контекст подставляется в system — пользователь не может его переопределить.
    1. Запрос модерируется ДО вызова RAG (если оставили эту фичу).
    1. Данные только из Object Storage — доверенный источник.
    1. API-ключ YandexGPT хранится в .env — не в коде.